In [197]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [198]:
monthly_emissions_1990_1996 = pd.read_csv("./monthly_emissions_1990_96.csv", index_col=[0])
monthly_emissions_1996_2024 = pd.read_csv("/Users/nicholasgeiser/Documents/Erdos/monthly-emissions-1996-2024-final.csv", index_col=[0])

In [201]:
# Convert from metric tons to short tons (for 1990-1996 data)
monthly_emissions_1990_1996['monthly_emissions'] = monthly_emissions_1990_1996['monthly_emissions']*1.10231

In [202]:
# For 1996-2024 data, convert to datetime
monthly_emissions_1996_2024['date'] = pd.to_datetime(monthly_emissions_1996_2024['date'])

# Extract month and year
monthly_emissions_1996_2024['month'] = monthly_emissions_1996_2024['date'].dt.month
monthly_emissions_1996_2024['year'] = monthly_emissions_1996_2024['date'].dt.year

# Delete date column
monthly_emissions_1996_2024 = monthly_emissions_1996_2024.drop(columns=['date'])

In [203]:
# Drop rows containing data for Alaska, Hawaii, and Puerto Rico
monthly_emissions_1996_2024 = monthly_emissions_1996_2024.drop(monthly_emissions_1996_2024.loc[monthly_emissions_1996_2024['State'].isin(['AK','HI','PR'])].index)

In [204]:
# Rename columns
monthly_emissions_1990_1996 = monthly_emissions_1990_1996.rename(columns={'co2_emissions':'monthly_emissiosn'})
monthly_emissions_1996_2024 = monthly_emissions_1996_2024.rename(columns={'State':'state', 'total_co2':'monthly_emissions'})

In [213]:
monthly_emissions_1990_1996.groupby('state')['monthly_emissions'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
AL,336.0,16.314122,24.586968,0.0,0.006616,0.185028,30.880620,78.075058
AR,336.0,6.417203,8.916840,0.0,0.015277,0.976025,12.092936,28.492972
AZ,336.0,10.051744,13.530127,0.0,0.045534,1.229159,20.722631,42.233621
CA,336.0,11.233225,15.030816,0.0,0.584575,2.229022,20.453583,54.436672
CO,336.0,8.999324,13.394077,0.0,0.000908,0.507260,18.052086,37.564401
CT,336.0,2.205100,2.601293,0.0,0.043512,1.247807,3.495203,12.157460
DC,336.0,0.065757,0.100257,0.0,0.000000,0.000000,0.113728,0.449877
DE,336.0,1.828539,2.168107,0.0,0.255286,0.853860,2.520061,8.533049
FL,336.0,24.660688,27.396948,0.0,3.888758,12.566826,38.146568,114.121404


In [212]:
monthly_emissions_1996_2024.groupby('state')['monthly_emissions'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
AL,360.0,5.945614e+06,1.830995e+06,1.246883e+06,4.847131e+06,6.160488e+06,7.289413e+06,9.658032e+06
AR,336.0,2.766459e+06,6.856993e+05,1.205562e+06,2.240457e+06,2.766734e+06,3.232378e+06,4.416156e+06
AZ,336.0,4.311630e+06,1.053314e+06,1.819085e+06,3.628626e+06,4.265543e+06,4.991716e+06,6.688225e+06
CA,336.0,3.019632e+06,9.902679e+05,8.664715e+05,2.297386e+06,3.018066e+06,3.691268e+06,5.654366e+06
CO,336.0,3.397356e+06,5.422328e+05,1.478247e+06,3.072296e+06,3.456909e+06,3.748682e+06,4.498211e+06
CT,336.0,7.206817e+05,1.825810e+05,3.017572e+05,5.882700e+05,6.989219e+05,8.170551e+05,1.450077e+06
DC,336.0,6.228494e+03,1.736621e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.975630e+03,1.181099e+05
DE,336.0,3.829412e+05,1.938501e+05,2.704849e+04,2.290495e+05,3.669981e+05,5.250093e+05,8.600581e+05
FL,360.0,9.489935e+06,2.862063e+06,1.168217e+06,8.367076e+06,9.623350e+06,1.144362e+07,1.486491e+07


In [206]:
# Merge datasets together
monthly_emissions_1990_2024 = pd.merge(monthly_emissions_1990_1996, monthly_emissions_1996_2024, 
                                       how='outer',
                                       on=['year','state','month'],
                                       suffixes=('_1990_1996','_1996_2024'))

In [207]:
monthly_emissions_1990_2024['monthly_emissions'] = monthly_emissions_1990_2024['monthly_emissions_1996_2024'].combine_first(
    monthly_emissions_1990_2024['monthly_emissions_1990_1996']
)

In [208]:
monthly_emissions_1990_2024 = monthly_emissions_1990_2024.drop(columns=['monthly_emissions_1990_1996', 'monthly_emissions_1996_2024'])

In [210]:
monthly_emissions_1990_1996

,year,month,state,monthly_emissions
0,1996,1,MD,1.569126
1,1996,2,MD,1.569126
2,1996,3,MD,1.569126
3,1996,4,MD,1.569126
4,1996,5,MD,1.569126
...,...,...,...,...
16459,1990,8,WY,0.000000
16460,1990,9,WY,8.733058
16461,1990,10,WY,8.733058
16462,1990,11,WY,17.466116


In [209]:
monthly_emissions_1990_2024

,year,month,state,monthly_emissions
0,1990,1,AL,0.000
1,1990,1,AL,0.000
2,1990,1,AL,0.000
3,1990,1,AL,0.000
4,1990,2,AL,0.000
...,...,...,...,...
32923,2024,8,WY,3392821.201
32924,2024,9,WY,2876132.311
32925,2024,10,WY,2761315.645
32926,2024,11,WY,2789809.785
